In [2]:
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.utils import np_utils
#from sklearn.cross_validation import StratifiedKFold

Using TensorFlow backend.
/anaconda3/envs/benchmarking/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/benchmarking/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/benchmarking/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/benchmarking/lib/pytho

In [2]:
PATH = os.getcwd()
print (PATH)

/Volumes/ADATA_HV620/indepented_projects/Classification-of-Hyperspectral-Image


In [3]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])


In [3]:
# Global Variables
#windowSize = 5
#numPCAcomponents = 30
#testRatio = 0.25

# Load Training Dataset

In [4]:
X_train = np.load("X_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("y_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [5]:
# Reshape into (numberofsamples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))

In [6]:
# convert class labels to on-hot encoding
y_train = np_utils.to_categorical(y_train)

In [7]:
# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

(30, 5, 5)


In [8]:
# number of filters
C1 = 3*numPCAcomponents

In [9]:
# Define the model
model = Sequential()

model.add(Conv2D(C1, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(3*C1, (3, 3), activation='relu'))
model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dense(6*numPCAcomponents, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=15)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
29685/29685 [==============================] - 7s 235us/step - loss: 2.4806 - acc: 0.2293
Epoch 2/15
29685/29685 [==============================] - 7s 221us/step - loss: 1.8380 - acc: 0.4680
Epoch 3/15
29685/29685 [==============================] - 7s 222us/step - loss: 1.3506 - acc: 0.5984
Epoch 4/15
29685/29685 [==============================] - 7s 242us/step - loss: 1.0505 - acc: 0.6706
Epoch 5/15
29685/29685 [==============================] - 7s 237us/step - loss: 0.8753 - acc: 0.7153
Epoch 6/15
29685/29685 [==============================] - 6s 202us/step - loss: 0.7567 - acc: 0.7478
Epoch 7/15
29685/29685 [==============================] - 6s 201us/step - loss: 0.6826 - acc: 0.7710
Epoch 8/15
29685/29685 [==============================] - 6s 213us/step - loss: 0.6224 - acc: 0.7878
Epoch 9/15
29685/29685 [==============================] - 8s 253us/step - loss: 0.5679 - acc: 0.8062
Epoch 10/15
29685/29685 [==================

In [12]:
import h5py
from keras.models import load_model

In [13]:
model.save('my_model' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio) + '.h5')